In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [2]:
dataset = pd.read_csv('IBM_2006-01-01_to_2018-01-01.csv')
dataset = dataset.fillna(dataset.mean())

In [3]:
dataset.describe()

,Open,High,Low,Close,Volume
count,3020.000000,3020.000000,3020.000000,3020.000000,3.020000e+03
mean,145.515545,146.681738,144.471597,145.617278,5.773301e+06
std,37.548726,37.613446,37.471433,37.529387,3.192831e+06
min,72.740000,73.940000,69.500000,71.740000,2.542560e+05
25%,116.407500,117.765000,115.500000,116.525000,3.622681e+06
50%,149.605000,150.330000,148.425000,149.315000,4.928852e+06
75%,178.437500,179.762500,177.320000,178.685000,6.965014e+06
max,215.380000,215.900000,214.300000,215.800000,3.077428e+07


In [4]:
dataset.head()

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM


In [5]:
dataset.tail()

,Date,Open,High,Low,Close,Volume,Name
3015,2017-12-22,151.82,153.00,151.50,152.50,2990583,IBM
3016,2017-12-26,152.51,153.86,152.50,152.83,2479017,IBM
3017,2017-12-27,152.95,153.18,152.61,153.13,2149257,IBM
3018,2017-12-28,153.20,154.12,153.20,154.04,2687624,IBM
3019,2017-12-29,154.17,154.72,153.42,153.42,3327087,IBM


In [6]:
training_set = dataset.iloc[0:3000, 1:2].values
training_set

array([[ 82.45],
       [ 82.2 ],
       [ 81.4 ],
       ...,
       [152.35],
       [152.81],
       [153.65]])

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
sc_training_set = sc.fit_transform(training_set)
sc_training_set

array([[0.06807347],
       [0.06632081],
       [0.06071228],
       ...,
       [0.55811834],
       [0.56134324],
       [0.56723219]])

In [8]:
# Create a data structure with 120 timesteps and 1 output
X_train = []
y_train = []
for i in range(120, 3000):
    X_train.append(sc_training_set[i-120:i,0])
    y_train.append(sc_training_set[i,0])

In [9]:
# convert them from lists to numpy arrays to train the estimator later
X_train = np.array(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.array(y_train)

In [10]:
# Import the following libraries to build RNN architecture
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [11]:
# Initialize RNN, 6 LSTM layers
estimator = Sequential()

estimator.add(LSTM(units=100,return_sequences=True,input_shape=(X_train.shape[1],1)))
estimator.add(Dropout(0.2))

estimator.add(LSTM(units=100,return_sequences=True))
estimator.add(Dropout(0.2))

estimator.add(LSTM(units=100,return_sequences=True))
estimator.add(Dropout(0.2))

estimator.add(LSTM(units=100,return_sequences=True))
estimator.add(Dropout(0.2))

estimator.add(LSTM(units=100,return_sequences=True))
estimator.add(Dropout(0.2))

# final LSTM layer
estimator.add(LSTM(units = 100))
estimator.add(Dropout(0.2))

# add output layer
estimator.add(Dense(units = 1))

In [12]:
# compiling the RNN
estimator.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [13]:
# Fit RNN to training set
estimator.fit(x=X_train, y=y_train, batch_size=32, epochs=10)

Epoch 1/10
2880/2880 [==============================] - 286s 99ms/step - loss: 0.0261
Epoch 2/10
2880/2880 [==============================] - 245s 85ms/step - loss: 0.0047
Epoch 3/10
2880/2880 [==============================] - 248s 86ms/step - loss: 0.0043
Epoch 4/10
2880/2880 [==============================] - 260s 90ms/step - loss: 0.0041
Epoch 5/10
2880/2880 [==============================] - 256s 89ms/step - loss: 0.0038
Epoch 6/10
2880/2880 [==============================] - 249s 87ms/step - loss: 0.0037
Epoch 7/10
2880/2880 [==============================] - 247s 86ms/step - loss: 0.0029
Epoch 8/10
2880/2880 [==============================] - 248s 86ms/step - loss: 0.0025
Epoch 9/10
2880/2880 [==============================] - 259s 90ms/step - loss: 0.0028
Epoch 10/10
2880/2880 [==============================] - 251s 87ms/step - loss: 0.0027


In [14]:
# making predictions with test set, the entire dataset
test_set = dataset.iloc[:,1:2].values
test_set

array([[ 82.45],
       [ 82.2 ],
       [ 81.4 ],
       ...,
       [152.95],
       [153.2 ],
       [154.17]])

In [15]:
sc_test_set = sc.fit_transform(test_set)

In [16]:
inputs = []
for i in range(120, 3020):
    inputs.append(sc_test_set[i-120:i, 0])

In [17]:
inputs = np.array(inputs)
inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))

In [18]:
predicted_stock_price = estimator.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [19]:
predicted_stock_price

array([[ 79.85224],
       [ 79.72834],
       [ 79.63535],
       ...,
       [155.14008],
       [154.87585],
       [154.57588]], dtype=float32)

In [20]:
# Visualizing the results
plt.plot(test_set, color = 'green', label = 'Actual stock prices')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted stock prices')
plt.title('Stock Price Prediction from January 2006 to December 2017')
plt.xlabel('Time(Days)')
plt.ylabel('Stock Price ($)')
plt.legend()
plt.show()